**1)Preprocessing:**

In [ ]:
!pip install transformers tensorflow numpy scikit-learn

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

**Collect the Best Examples on the basis of Similarity Score using BERT Model**

In [ ]:
import pandas as pd
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Method/ChineseMethod.csv')
from sklearn.model_selection import train_test_split
# Split the data into training (80%) and testing (20%) sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
df_train.head()

,Method,Description
1513,update,使用指定的字节更新当前校验和。
479,deleteFile,删除文件。
3965,getView,view属性标识 \n AbstractView生成事件的 \n...
3932,getPublicId,如果指定，则与实体关联的公共标识符，否则为 \n null 。
2258,getMimeTypes,返回此对象支持的MIME类型的 \n DataFlavor对象的数组。


In [ ]:
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:

# Define the number of rows to process in each batch
batch_size = 1000

# Initialize an empty list to store similarity scores for the entire DataFrame
all_similarity_scores = []

# Iterate through the df_train DataFrame in batches
for start in range(0, len(df_train), batch_size):
    # Get a batch of rows (e.g., 1000 rows)
    batch_df = df_train.iloc[start:start + batch_size]

    # Initialize an empty list to store similarity scores for this batch
    batch_similarity_scores = []

    # Iterate through the rows in the batch and calculate similarity scores
    for index, row in batch_df.iterrows():
        # Tokenize and encode the sentences
        encoded_descriptions = tokenizer(row["Description"], padding=True, truncation=True, return_tensors="tf")
        encoded_method_names = tokenizer(row["Method"], padding=True, truncation=True, return_tensors="tf")

        # Generate embeddings for the sentences
        embed_descriptions = model(encoded_descriptions).last_hidden_state[:, 0, :].numpy()
        embed_method_names = model(encoded_method_names).last_hidden_state[:, 0, :].numpy()

        # Calculate cosine similarity between embeddings
        similarity_score = cosine_similarity(embed_descriptions, embed_method_names)[0][0]

        # Append the similarity score to the batch list
        batch_similarity_scores.append(similarity_score)

    # Add the batch similarity scores to the list of all similarity scores
    all_similarity_scores.extend(batch_similarity_scores)

# Add all similarity scores to the df_train DataFrame
df_train["similarity_score"] = all_similarity_scores

# Print the df_train DataFrame with similarity scores
print(df_train)

            Method                                        Description  \
1513        update                                    使用指定的字节更新当前校验和。   
479     deleteFile                                              删除文件。   
3965       getView  view属性标识 \n               AbstractView生成事件的 \n...   
3932   getPublicId      如果指定，则与实体关联的公共标识符，否则为 \n               null 。   
2258  getMimeTypes  返回此对象支持的MIME类型的 \n               DataFlavor对象的数组。   
...            ...                                                ...   
3444       setClob                将指定参数设置为 \n               Reader对象。   
466      normalize                              返回此路径的路径，其中删除了冗余名称元素。   
3092    getBoolean  在Java编程语言中以 \n               boolean检索指定的JDBC ...   
3772       warning                                            收到警告通知。   
860       hashCode                                         此日期时间的哈希码。   

      similarity_score  
1513          0.798680  
479           0.818419  
3965          0.865146  
3932          0.768480 

In [ ]:
# Define the scoring function
def calculate_method_name_score(method_name):
    score = 0

    # Rule 1: Use of Verb-Noun Pairs
    if any(word.isalpha() and word[0].islower() for word in method_name.split()):
        score += 0.2

    # Rule 2: CamelCase Formatting
    if method_name.isidentifier():
        score += 0.2

    # Rule 3: Specificity
    if len(method_name) <= 20:  # Adjust the length threshold as needed
        score += 0.2

    # Rule 4: Avoiding Java Reserved Words (you can define a list of reserved words to check against)
    reserved_words = ["public", "private", "static", "int", "abstract", "assert", "boolean", "break", "byte", "case",
                      "catch", "char", "class", "const", "continue", "default", "do", "double", "else", "enum",
                      "extends", "final", "finally", "float", "for", "goto", "if", "implements", "import",
                      "instanceof", "interface", "long", "native", "new", "package", "protected", "public",
                      "return", "short", "static", "strictfp", "super", "switch", "synchronized", "this", "throw",
                      "throws", "transient", "try", "void", "volatile", "while"]  # Define the list
    if method_name not in reserved_words:
        score += 0.2

    # Rule 5: Avoiding Special Characters and Spaces
    if all(c.isalnum() or c == '_' for c in method_name):
        score += 0.2

    return score

# Add the "EvlScore" column to the DataFrame
df_train["EvlScore"] = df_train["Method"].apply(calculate_method_name_score)

# Print the DataFrame with the "EvlScore" column
print(df_train)


            Method                                        Description  \
1513        update                                    使用指定的字节更新当前校验和。   
479     deleteFile                                              删除文件。   
3965       getView  view属性标识 \n               AbstractView生成事件的 \n...   
3932   getPublicId      如果指定，则与实体关联的公共标识符，否则为 \n               null 。   
2258  getMimeTypes  返回此对象支持的MIME类型的 \n               DataFlavor对象的数组。   
...            ...                                                ...   
3444       setClob                将指定参数设置为 \n               Reader对象。   
466      normalize                              返回此路径的路径，其中删除了冗余名称元素。   
3092    getBoolean  在Java编程语言中以 \n               boolean检索指定的JDBC ...   
3772       warning                                            收到警告通知。   
860       hashCode                                         此日期时间的哈希码。   

      similarity_score  EvlScore  
1513          0.798680       1.0  
479           0.818419       1.0  
3965          0.86

In [ ]:
# Save the new DataFrame to a CSV file
df_train.to_csv('/content/drive/MyDrive/Method/ChineseDSforGPT/DSTrain.csv', index=False)
df_test.to_csv('/content/drive/MyDrive/Method/ChineseDSforGPT/DSTest.csv', index=False)

**Sort the Examples to get Top 1000 examples**

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Method/ChineseDSforGPT/DSTrain.csv')

# Sort the DataFrame by similarity_score and EvlScore in descending order
df_sorted = df.sort_values(by=["similarity_score", "EvlScore"], ascending=[False, False])

# Get the top 1000 pairs
QExpDS = df_sorted.head(100)
QExpDS.to_csv('/content/drive/MyDrive/Method/ChineseDSforGPT/DSBestExamples.csv', index=False)


**FineTunning**

In [ ]:
!pip install openai==0.28

**Here we write ChatGPT Funcation which will get prompt as input and resturn responce as output**

In [ ]:
import openai
import requests
import json
openai.api_key = 'provide your api key'
def get_chatgpt_top_four_responses(prompt_text):
    # Make a request to the OpenAI API for four completions
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",  # or "gpt-4-turbo", depending on your access
        prompt=prompt_text,
        max_tokens=200,  # Set your desired maximum token limit
        n=4  # Request four completions/responses
    )

    # Extract the generated responses
    chatgpt_responses = [choice['text'].strip() for choice in response['choices']]
    return chatgpt_responses


**Here we write the unique prompt with best examples get through the preprocessing **bold text** **

In [ ]:
prompt="""Prompt: Suggest a Java Method Name
Given a functional description of an operation, your task is to suggest a concise, descriptive method name that follows Java naming conventions. The method name should accurately reflect the action performed and the object it acts upon, using camelCase notation.
Here are best examples of method names that follow these guidelines:
1.	Functional Description: Initialize autoscrolling.
•	Method Name: initializeAutoscrolling
2.	Functional Description: Instantiate a bean.
•	Method Name: instantiate
3.	Functional Description: Write a byte.
•	Method Name: write
4.	Functional Description: Add a service.
•	Method Name: addService
5.	Functional Description: If the number of arguments in the specified constructor is non-zero and the class of the old instance explicitly declares an "equals" method, this method returns the value of old instance equals new instance.
•	Method Name: mutatesTo
6.	Functional Description: Validates this component.
•	Method Name: validate
7.	Functional Description: Returns whether the representation class for this data flavor is java.nio.CharBuffer or a subclass thereof.
•	Method Name: isRepresentationClassCharBuffer
8.	Functional Description: Dispatches an event.
•	Method Name: dispatchEvent
9.	Functional Description: Deprecated as of JDK version 1.1, replaced by is multiple mode.
•	Method Name: allowsMultipleSelections
10.	Functional Description: This method instructs the bean that it is OK to use the GUI.
•	Method Name: okToUseGui
11.	Functional Description: Dispatches an event to the active input method.
•	Method Name: dispatchEvent
12.	Functional Description: Bean context services listener callback, propagates event to all currently registered listeners and bean context services children. If this bean context service does not already implement this service itself.
•	Method Name: serviceAvailable
13.	Functional Description: Deprecated as of JDK version 1.1, replaced by get selected checkbox.
•	Method Name: getCurrent
14.	Functional Description: Removes a tile observer.
•	Method Name: removeTileObserver
15.	Functional Description: Checks if this window is showing on screen.
•	Method Name: isShowing
16.	Functional Description: Subclasses may implement this method to allow class data to be stored in the stream.
•	Method Name: annotateClass
17.	Functional Description: Returns the hashcode for this menu shortcut.
•	Method Name: hashCode
18.	Functional Description: Add a property change listener to the listener list.
•	Method Name: addPropertyChangeListener
19.	Functional Description: Removes the specified component from this layout.
•	Method Name: removeLayoutComponent
20.	Functional Description: Checks out a tile for writing.
•	Method Name: getWritableTile
21.	Functional Description: Removes an item listener.
•	Method Name: removeItemListener
22.	Functional Description: Deprecated as of JDK version 1.1, replaced by get visible amount.
•	Method Name: getVisible
23.	Functional Description: Notify the component to autoscroll.
•	Method Name: autoscroll
24.	Functional Description: Clones this object.
•	Method Name: clone
25.	Functional Description: Updates this canvas.
•	Method Name: update
26.	Functional Description: Returns the hit on the opposite side of the specified hit's caret.
•	Method Name: getVisualOtherHit
27.	Functional Description: Removes the specified window listener so that it no longer receives window events from this window.
•	Method Name: removeWindowListener
28.	Functional Description: Deprecated as of JDK version 1.1, replaced by transfer focus.
•	Method Name: nextFocus
29.	Functional Description: Analogous to java.lang.ClassLoader.getResourceAsStream. This method allows a bean context implementation to interpose behavior between the child component and underlying class loader.
•	Method Name: getResourceAsStream
30.	Functional Description: Deprecated as of JDK version 1.1, replaced by getClipBounds.
•	Method Name: getClipRect
31.	Functional Description: Resolves instances being deserialized into instances registered with CMM.
•	Method Name: readResolve
32.	Functional Description: Deprecated as of JDK version 1.1, not for public use in the future. This method is expected to be retained only as a package-private method.
•	Method Name: delItems
33.	Functional Description: Deprecated as of JDK version 1.1, replaced by set selected checkbox checkbox.
•	Method Name: setCurrent
34.	Functional Description: Gets the bean context services listener if any of the specified child.
•	Method Name: getChildBeanContextServicesListener
35.	Functional Description: Returns whether the representation class for this data flavor is java.nio.ByteBuffer or a subclass thereof.
•	Method Name: isRepresentationClassByteBuffer
36.	Functional Description: Write a portion of a string to the buffer.
•	Method Name: write
37.	Functional Description: Converts this font object to a string representation.
•	Method Name: toString
38.	Functional Description: Returns the device configuration associated with this Graphics2D.
•	Method Name: getDeviceConfiguration
39.	Functional Description: Gets the singleton instance of the system clipboard which interfaces with clipboard facilities provided by the native platform.
•	Method Name: getSystemClipboard
40.	Functional Description: This method is called by the current keyboard focus manager requesting that this key event dispatcher dispatch the specified event on its behalf.
•	Method Name: dispatchKeyEvent
41.	Functional Description: Enables or disables input method support for this component.
•	Method Name: enableInputMethods
42.	Functional Description: Returns the number of milliseconds this robot sleeps after generating an event.
•	Method Name: getAutoDelay
43.	Functional Description: This method is called by the AWT event dispatcher requesting that the current keyboard focus manager dispatch the specified event on its behalf.
•	Method Name: dispatchEvent
44.	Functional Description: Removes the specified window state listener so that it no longer receives window events from this window.
•	Method Name: removeWindowStateListener
45.	Functional Description: The setProperties method is part of the image consumer API which this class must implement to retrieve the pixels.
•	Method Name: setProperties
46.	Functional Description: Dispatches the event to the input method.
•	Method Name: dispatchEvent
47.	Functional Description: Removes a bean context membership listener.
•	Method Name: removeBeanContextMembershipListener
48.	Functional Description: Bean context services listener callback, propagates event to all currently registered listeners and bean context services children. If this bean context service does not already implement this service itself.
•	Method Name: serviceRevoked
49.	Functional Description: Updates the container.
•	Method Name: update
50.	Functional Description: This method returns a Graphics2D, but is here for backward compatibility.
•	Method Name: getGraphics
When suggesting method names, consider the following guidelines:
•	Use verb-noun pairs where appropriate to indicate actions on objects (e.g., findIndex).
•	Employ camelCase, starting with a lowercase letter and capitalizing the first letter of each subsequent word.
•	Be as specific as possible without making the name too lengthy.
•	Avoid using Java reserved words.
•	Do not use special characters or spaces.
Suggest four method names of given functional description based on these guidelines and best examples provided.
Functional Description: Searches for a specific element within a list and returns its index.
Your Suggested Method Name:?
"""

**Pass this Prompt to Chatgpt for fintuning**

In [ ]:
response = get_chatgpt_response(prompt)
print(response)

**3) Postprocessing**

In [ ]:
!pip install transformers tensorflow numpy scikit-learn

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertModel
from sklearn.metrics.pairwise import cosine_similarity
# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

**This is Reward Model which rank the responces accoruding to their similarity score **

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
# Assume tokenizer and model are previously defined and imported

def rank_method_names(data, original_method):
    # Create a DataFrame with the sample data
    df = pd.DataFrame(data, columns=["Method Name"])

    # Add the original method to the DataFrame
    df.loc[len(df)] = [original_method]

    # Initialize an empty list to store similarity scores
    similarity_scores = []

    # Iterate through the methods and calculate similarity scores with the original method
    for index, row in df.iterrows():
        if row["Method Name"] != original_method:
            # Tokenize and encode the sentences
            encoded_original_method = tokenizer(original_method, padding=True, truncation=True, return_tensors="tf")
            encoded_method = tokenizer(row["Method Name"], padding=True, truncation=True, return_tensors="tf")

            # Generate embeddings for the sentences
            embed_original_method = model(encoded_original_method).last_hidden_state[:, 0, :].numpy()
            embed_method = model(encoded_method).last_hidden_state[:, 0, :].numpy()

            # Calculate cosine similarity between embeddings
            similarity_score = cosine_similarity(embed_original_method, embed_method)[0][0]

            # Append the similarity score to the list
            similarity_scores.append(similarity_score)
        else:
            # If comparing with itself, set similarity score to 1
            similarity_scores.append(1.0)

    # Add similarity scores to the DataFrame
    df["similarity_score"] = similarity_scores

    # Rank the DataFrame based on similarity scores in descending order
    ranked_df = df.sort_values(by="similarity_score", ascending=False)

    # Print the ranked responses in the desired format
    rankedResponse = "Feedback:In terms of semantic similarity scores with the original method name, the ranking of responses would be as follows:\n"
    for index, row in ranked_df.iterrows():
        rankedResponse += row["Method Name"] + " > "

    # Remove the last " > " and add a newline character at the end to format the output
    rankedResponse = rankedResponse.rstrip(" > ") + "\n"

    return rankedResponse




In [ ]:
data=["FontToStr", "toStringFont","fontToText","getFontAsString"]
ActualMethod="toString"
prompt=rank_method_names(data, ActualMethod)
print(prompt)

Feedback:In terms of semantic similarity scores with the original method name, the ranking of responses would be as follows:
toString > toStringFont > FontToStr > getFontAsString > fontToText



**Here we are doing PostProcessing**

In [ ]:
dataset = pd.read_csv('Dataset.csv')

# Assuming the column for functional descriptions is named 'Functional Description'
for index, row in dataset.iterrows():
    functional_description = row['Functional Description']
    response_methods = []
    # Here we get funcation Description one by one and put it into the following prompt
    prompt = f"Suggest the a Method Name along with three alternative Method names for the following Functional Description: {functional_description}"
    # This prompt will pass to the ChatGPT funcation , it will get 4 Method Name as responce
    response = get_chatgpt_response(prompt)
    # Format the response
    formatted_response = f"Suggested Methods are: {response}"
    methods = response.split(", ")
    # Here we insert these responces in to a list for further process
    response_methods.extend(methods)
    original_method=row['Method Name']
    # Now rank these responces using similarity score , rank_method_names will retunr a prompt with ranked responces
    # Like this In terms of similarity scores with the original method name, the ranking of responses would be as follows:
    # getAccessibleText > getAccessibleText > fetchAssociatedAccessibleText > retrieveAccessibleText > obtainAccessibleObjectText
    rankedResponsePrompt = rank_method_names(response_methods, original_method)
    # Give rank responces to chat_get_chatgpt_response
    get_chatgpt_response(rankedResponsePrompt)
    # This Process will repeate on all funcational description one by one

